# Data Journalism Lesson 7: Working with dates

Learn how to make a "date" into a real date you can work with using Python.

In [ ]:
# Setup code for the notebook
import pandas as pd
from IPython.display import display, HTML

# Helper functions for grading exercises
def display_feedback(correct, message_correct, message_incorrect):
    if correct:
        display(HTML(f'<div style="background-color: #dff0d8; padding: 10px; border-radius: 5px;"><strong>Correct!</strong> {message_correct}</div>'))
    else:
        display(HTML(f'<div style="background-color: #f2dede; padding: 10px; border-radius: 5px;"><strong>Not quite!</strong> {message_incorrect}</div>'))


def check_datetime_conversion(series, expected):
    try:
        pd.testing.assert_series_equal(series, expected, check_names=False)
        display_feedback(True, 'Excellent! The date column is now in the correct datetime format.', '')
    except AssertionError:
        display_feedback(False, '', 'Check your conversion function and the input column.')

def check_floor_year_agg(df_agg, expected_cols=['floor_year', 'n']):
    correct = isinstance(df_agg, pd.DataFrame) and list(df_agg.columns) == expected_cols and pd.api.types.is_integer_dtype(df_agg['n'])
    display_feedback(correct, 'Correctly aggregated crashes by year!', 'Check your grouping, aggregation, and final column names.')

In [ ]:
state = "Minnesota"

dataurl = f"../_static/plane-crashes/{state.lower()}.csv"

crashes = pd.read_csv(dataurl)
nrows = len(crashes)
ncolumns = len(crashes.columns)

# Pre-calculate totals for the results section
crashes_temp = crashes.copy() # Avoid modifying original df used in exercises
crashes_temp['clean_date'] = pd.to_datetime(crashes_temp['first_bad_date'], format='%m/%d/%Y', errors='coerce')
crashes_temp['floor_year'] = crashes_temp['clean_date'].dt.year
totals_df = crashes_temp.groupby('floor_year').size().reset_index(name='n')
total_2024 = totals_df[totals_df['floor_year'] == 2024]['n'].iloc[0] if 2024 in totals_df['floor_year'].values else 0

clean_date_expected = pd.to_datetime(crashes['first_bad_date'], format="%m/%d/%Y", errors='coerce').rename('clean_date')
clean_date2_expected = pd.to_datetime(crashes['second_bad_date'], format="%d-%m-%Y", errors='coerce').rename('clean_date2')


In [ ]:
# Glue variables for use in markdown
from myst_nb import glue

glue("state_full_name", state, display=False)
glue("nrows", nrows, display=False)
glue("total_2024", f"{total_2024}", display=False)

## The Goal

In this lesson, you'll learn how to work with dates in Python using the pandas library, a critical skill for data analysis. By the end of this tutorial, you'll understand how to convert character strings into proper date formats. You'll practice reformatting dates using `pd.to_datetime()` and learn how to use datetime properties and methods (like `.dt.year` or `.dt.floor()`) to group dates. These skills will enable you to perform time-based analyses and answer questions about trends over time in your datasets.

## What is Data Journalism?

Yanqi Xu, a reporter at the Flatwater Free Press in Omaha, Neb., started her road into data journalism through interactives. In grad school at the University of Missouri, it was multimedia and charts that moved and that you could click on, which hatched her interest in code and data.

The real thunderclap moment for her came in a data journalism class -- her "third time trying to learn R."

"And I realized oh, my God, like, there are so many problems I will be able to solve if I know how to, you know, manipulate strings, for example, clean text, and just use that, make a pivot table," she said.

One of Xu's first stories was based largely around dates. The basic premise of the story: [Members of the parole board -- appointed by the governor -- weren't showing up for work.](https://flatwaterfreepress.org/the-full-nebraska-parole-board-appears-at-less-than-half-of-hearings-when-members-miss-votes-fewer-get-paroled/) In fact, the full board appeared in only half of meetings. The other half? Appointees making $85,000 in taxpayer-funded salaries didn't show.

Since the story ran, legislators have passed new rules requiring them to be present.

The story started with spreadsheets of voting records made by people. What I mean by that is each row wasn't a complete record. A human could look at the spreadsheet, see a date on the first row, and interpret that the date applied to all rows. Same for other bits of information. But as you've learned -- for data to be analyzed, each row must contain complete, and consistent data.

Xu spent hours reformatting information, transforming the structure of the data, and melding it back together. One of the first jobs was standardizing dates.

The tools she used are the same you'll learn here. And don't worry -- she doesn't remember the exact details of each date function either. She does what lots of people do: Make a cheatsheet.

"I actually have all the strings, common formats, saved in my notes that, you know, it's my cheatsheet almost," she told me. "So I save it, and I don't have to remember anything. And the truth is, I'm not going to remember everything."

## The Basics

One of the most frustrating things in data is working with dates. Everyone has a different opinion on how to record them, and every software package on the planet has to sort it out. Dealing with it can be a little ... confusing. And every dataset has something new to throw at you. So consider this an introduction.

But know this: If your story questions have date-based answers, you're going to 1) need a column that has date information in it and 2) that date information needs to be formatted correctly so your software can read and interpret the date as a date.

We're going to do this using the `pandas` library, which builds on Python's built-in `datetime` capabilities. Pandas provides powerful and flexible tools for handling time series data.

## Working with Dates in Pandas

First, we'll import `pandas`.

In [ ]:
import pandas as pd

For this, we're going to look at a dataset from the National Transportation Safety Board, which is a database of all airplane crashes since 2019. The irony here is that this dataset, when you download it from the NTSB, has properly formatted dates. Every dataset I could find that had bad dates was too large to efficiently use in this tutorial.

Planes crash -- and small planes crash far more often than big planes. Commercial airline crashes are exceedingly rare -- years can go by in the United States between major airline crashes. Every state has a number of small plane crashes every year. If you work as a breaking news reporter, you will very likely have to cover a small plane crash at some point. Having this data, or knowing where it is to get it quickly on deadline, can add depth and context to a breaking news story about a pilot injured in a botched landing.

Questions you can ask: How many crashes are there in a year? Is this year worse or better than last year? Is there a seasonal element to crashes in your state? I live in the Midwest, where spring winds are no joke and early summer thunderstorms can get pretty wild. Warm winter states might get an influx of older pilots who might need to hang it up. You won't know until you look.

Let's load up our data as we've been doing all along.

In [ ]:
# Read the CSV data from the dataurl
crashes_df = pd._____("../_static/plane-crashes/minnesota.csv")
print(crashes_df.head())


And let's look at it quick using `.info()`.

In [ ]:
# Display concise summary of the DataFrame
crashes_df.____()

At the bottom of the data summary, you'll see columns like `first_bad_date` and `second_bad_date`. Notice their `Dtype` (data type) is `object`, which usually means strings in pandas. We have to fix that.

The primary way to convert columns to datetime objects in pandas is using `pd.to_datetime()`. This function is quite flexible.

### Method 1: Specifying the Format

If you know the exact format of your date strings, you can tell `pd.to_datetime()` using the `format` argument. This is often the most reliable way.

Here are a few common format codes:

| Code | Meaning                                        | Example |
|------|------------------------------------------------|---------|
| %Y   | Year with century (e.g., 2023)                 | `2023`  |
| %y   | Year without century (00-99)                   | `23`    |
| %m   | Month as a zero-padded decimal number (01-12)  | `01`    |
| %d   | Day of the month as a zero-padded decimal (01-31) | `19`    |
| %b   | Abbreviated month name (Jan, Feb, ...)         | `Jan`   |
| %B   | Full month name (January, February, ...)       | `January`|
| %H   | Hour (24-hour clock) as zero-padded decimal (00-23) | `14`    |
| %I   | Hour (12-hour clock) as zero-padded decimal (01-12) | `02`    |
| %M   | Minute as zero-padded decimal (00-59)          | `05`    |
| %S   | Second as zero-padded decimal (00-59)          | `09`    |
| %p   | Locale's equivalent of either AM or PM.        | `PM`    |
| %a   | Abbreviated weekday name (Sun, Mon, ...)       | `Mon`   |
| %A   | Full weekday name (Sunday, Monday, ...)        | `Monday`|

Look at the `first_bad_date` column. It has dates like `1/19/2023`. The format is month, day, four-digit year, separated by slashes.

### Exercise 1: Formatting the date using `format`

Use `pd.to_datetime()` with the correct `format` string to convert the `first_bad_date` column. We'll create a new column `clean_date` to store the result. It's good practice not to overwrite original data immediately.

*(Hint: You need codes for month, day, and 4-digit year, separated by slashes)*

In [ ]:
# Create a new column 'clean_date' by converting 'first_bad_date'
# using pd.to_datetime and specifying the format.
crashes_df['clean_date'] = pd.to_datetime(crashes_df['first_bad_date'], format="____", errors='coerce')

# Display the first 5 rows with the new column
print(crashes_df[['first_bad_date', 'clean_date']].head())

# Check the conversion
check_datetime_conversion(crashes_df['clean_date'], clean_date_expected)

And just like that, `clean_date` is now a datetime column (you can verify with `crashes_df.info()` again if you like). That means we can do date-based operations with it – things like grouping by month or calculating the time difference between crashes.

You can almost read the format code: `%m` (month) SLASH `%d` (day) SLASH `%Y` (4-digit year). You can remix that as needed. If you had a date like `20021212`, the format would be `format="%Y%m%d"`. Have dashes instead of slashes? Just use dashes in the format string: `format="%Y-%m-%d"`. Got a date like `January 1, 2019`? That's `format="%B %d, %Y"`. While specifying the format requires you to know it, it's very precise.

### Method 2: Letting Pandas Infer the Format

`pd.to_datetime()` is also quite good at *inferring* common date formats automatically if you don't provide the `format` argument.

Let's try this on the `first_bad_date` column again. Since the format `MM/DD/YYYY` is very common, pandas can usually figure it out.

### Exercise 2: Inferring the format

Use `pd.to_datetime()` *without* the `format` argument to convert `first_bad_date`.

In [ ]:
# Convert 'first_bad_date' letting pandas infer the format
crashes_df['clean_date_inferred'] = pd.to_datetime(crashes_df[____], errors='coerce')

# Display the first 5 rows with the new column
print(crashes_df[['first_bad_date', 'clean_date_inferred']].head())

# Check the conversion
check_datetime_conversion(crashes_df['clean_date_inferred'], clean_date_expected)

That's less code, which is often good. However, relying on inference can sometimes lead to unexpected results if dates are ambiguous (like `01/02/03` - is that Jan 2, 2003, Feb 1, 2003, or Feb 3, 2001?) or if the format is unusual. Specifying the format is generally safer if you know it.

### Exercise 3: Try it again with a different format

Now look at the `second_bad_date` column. You might see dates like `30-8-2024`. What format is that?

Use `pd.to_datetime()` to parse the dates in `second_bad_date`. You can either specify the format explicitly or try letting pandas infer it (though specifying is safer here).

In [ ]:
# Convert 'second_bad_date' to datetime.
# Choose either format inference or specify the format string.
crashes_df['clean_date2'] = pd.to_datetime(crashes_df[____], format="____", errors='coerce') # Option 1: Specify format
# crashes_df['clean_date2'] = pd.to_datetime(crashes_df['second_bad_date'], errors='coerce') # Option 2: Infer format

# Display the first 5 rows with the new column
print(crashes_df[['second_bad_date', 'clean_date2']].head())

# Check the conversion
check_datetime_conversion(crashes_df['clean_date2'], clean_date2_expected)

## More with dates

Once you have a proper datetime column, pandas provides a convenient `.dt` accessor to extract various components or perform date-related operations.

Let's start with the question: How many plane crashes happen each year?

### Exercise 4: Aggregating by Year

We need to group our data by year and count the number of crashes in each year.

1.  **Ensure you have a clean date column**: We'll use the `clean_date` column we created in Exercise 1 (or `clean_date_inferred` from Exercise 2).
2.  **Extract the year**: Use the `.dt.year` accessor on the datetime column to get the year for each crash.
3.  **Group and count**: Group the DataFrame by the extracted year and count the number of rows in each group.

We'll use the `.groupby()` method followed by `.size()` (which counts rows per group) and `.reset_index(name='n')` to turn the result back into a DataFrame with columns for the year and the count (`n`).

In [ ]:
# Extract the year into a new column
crashes_df['floor_year'] = crashes_df['clean_date'].____

# Group by year and count crashes
yearly_counts = crashes_df.groupby(____).size().reset_index(name='n')

# Display the result
print(yearly_counts)

# Check the aggregation result
check_floor_year_agg(yearly_counts)

A paragraph you can write about plane crashes: In {glue:text}`state_full_name`, there's been {glue:text}`nrows` airplane accidents investigated by the National Transportation Board since 2019. So far, in 2024, there's been {glue:text}`total_2024`.

You could go further, of course. How many deaths? How many injuries? How does this year compare to last? What kind of pace is this year on? Has there been other accidents at this airport since 2019? **These are all questions you can answer, right now, with what you know.** (Combined with skills from previous lessons on filtering, aggregation, etc.)

## The Recap

Throughout this lesson, you've learned how to handle dates in Python using the pandas library. You've practiced using `pd.to_datetime()` to parse dates, both by specifying the exact format and by letting pandas infer it. You've also learned about the `.dt` accessor, using `.dt.year` to extract the year component for aggregation. Remember, working with dates is a common challenge in data journalism, and these skills will help you analyze time-based patterns in your data more effectively. As you continue to work with different datasets, you'll encounter various date formats, and the techniques you've learned here will serve as a foundation for handling these challenges.

## Terms to Know

- **pandas**: A fundamental Python library for data manipulation and analysis.
- **DataFrame**: A primary pandas data structure, representing data in a tabular (rows and columns) format.
- **Series**: A one-dimensional labeled array in pandas, representing a single column in a DataFrame.
- **`pd.to_datetime()`**: A pandas function used to convert arguments (like Series, lists, or strings) to datetime objects.
- **`format` argument**: An argument in `pd.to_datetime()` used to specify the exact format of input date strings using codes like `%Y`, `%m`, `%d`.
- **Format Inference**: The ability of `pd.to_datetime()` to automatically detect the format of common date strings.
- **`errors='coerce'`**: An argument for `pd.to_datetime()` that turns unparseable dates into `NaT` (Not a Time) instead of raising an error.
- **datetime object**: A data type in pandas (and Python) specifically designed to represent dates and times, allowing for time-based calculations and operations.
- **`.dt` accessor**: A special accessor for pandas Series with datetime values, providing convenient access to date/time properties (like `.dt.year`, `.dt.month`, `.dt.day`, `.dt.weekday`) and methods (like `.dt.floor()`, `.dt.strftime()`).
- **`object` dtype**: A pandas data type that often indicates strings, but can hold mixed types.
- **`datetime64[ns]`**: The typical pandas data type for datetime objects, representing nanosecond precision.